In [5]:
import pandas as pd 
import sqlite3 as sql3

class CreateMinardDB:
    def __init__(self):
        """This function will load the origianl column names from the .txt file"""
        
        self.file_names ='minard.txt'
        file_path = f"data/{self.file_names}"
        
        # Read the .txt file
        with open(file_path) as f: 
            lines = f.readlines()
            column_names = lines[2].split()
            
        # Clean the column strings
        redundant_symbols = ['(','$',')',',']
        revised_col_names = []
        for col in column_names:
            for s in redundant_symbols: 
                if s in col:
                    col = col.replace(s,"")
            revised_col_names.append(col)

        # Divide the columns by the information in the data
        print("Loading the columns >>>")
        col_city = revised_col_names[:3]
        col_temp = revised_col_names[3:7]
        col_troop =  revised_col_names[7:]

        self.column_names_city = col_city
        self.column_names_temp = col_temp
        self.column_names_troop = col_troop
        self.lines = lines
        
        
        print(f"city data: {col_city}")
        print(f"temperature data: {col_temp}")
        print(f"troop data: {col_troop}")
        
        
    def create_city_df(self):
        """Read the data regarding the city informations, 
        which includes longtitudes, latitudes, and the names of the city. """
        
        i = 6
        longtitudes, latitudes, cities = [], [], []

        while i <= 25: 
            lon, lat, city = self.lines[i].split()[:3]
            longtitudes.append(float(lon))
            latitudes.append(float(lat))
            cities.append(str(city))
            i += 1
        city_data = [longtitudes, latitudes, cities]

        # Create a Dataframe for city data
        city_df = pd.DataFrame()
        for col_name, data in zip(self.column_names_city, city_data):  
            # Name the columns and load the data simultaneously
            city_df[col_name] = data 
        return city_df
        

    def create_temp_df(self):
        """Read the data regarding the temperature information, which includes longtitudes, temperatures, the days between recording dates and the recording dates. """    
        i = 6 
        longtitudes, temperatures, days, dates= [], [], [], []

        while i <= 14:
            lont, temp, day = self.lines[i].split()[3:6]
            # The "date" colum in raw data was sperated into [month day] 
            # Here we are combining them back togeter into one cell
            date = self.lines[i].split()[6:8] 
            date = ' '.join(date)    
            if i == 10:
                date = 'Nov 24'
                
            longtitudes.append(float(lont))
            temperatures.append(int(temp))
            days.append(int(day))
            dates.append(str(date))
            i+=1      
        temp_data = [longtitudes, temperatures, days,  dates]
      
        # Create a Dataframe for temperature data
        temp_df = pd.DataFrame()
        for col_name, data in zip(self.column_names_temp, temp_data):  
            # Name the columns and load the data simultaneously
            temp_df[col_name] = data
        return temp_df
    
    def create_troop_df(self):
        """Read the data regarding the troop information, which includes longtitudes, temperatures, the days between recording dates and the recording dates. """    
        
        i = 6 
        lonps, latps, survivs, direcs, divisions = [], [], [], [], []

        while i <= 53:
            lonp, latp, surviv, direc, division = self.lines[i].split()[-5:]  
            lonps.append(float(lonp))
            latps.append(float(latp))
            survivs.append(int(surviv))
            direcs.append(direc)
            divisions.append(int(division))
            i+=1

        troop_data = [lonps, latps, survivs, direcs, divisions]

        # Create a Dataframe for troop data
        troop_df = pd.DataFrame()
        for col_name, data in zip(self.column_names_troop, troop_data):  
            # Name the columns and load the data simultaneously
            troop_df[col_name] = data 
        return troop_df
    
    def persist_df(self):
        """Save a DataFrame as a callable object in python without recomputing results"""
        
        self.create_city_df().to_parquet('city_df.parquet')
        self.create_temp_df().to_parquet('temp_df.parquet')
        self.create_troop_df().to_parquet('troop_df.parquet')
        
    
    def create_sql_db(self):
        """Create a SQL database named minard.db with the clean data"""
        
        db_name = "minard.db"
        connection = sql3.connect(f"data/{db_name}")
        
        # Send the SQL descriptions to SQLite database
        # Ensure that there's no plot with the same name we are going to create

        df_dict = {
            "cities": self.create_city_df(),
            "temperatures": self.create_temp_df(),
            "troops": self.create_troop_df()
        }
        for key, value in df_dict.items():
            value.to_sql(name=key, con=connection, index=False, if_exists="replace")
        print("\nFinished loading DataFrames into SQLite3!")
            
        ### Execute the SQL script
        connection.close()
        
minard_db = CreateMinardDB()
minard_db.persist_df()
minard_db.create_sql_db()


Loading the columns >>>
city data: ['lonc', 'latc', 'city']
temperature data: ['lont', 'temp', 'days', 'date']
troop data: ['lonp', 'latp', 'surviv', 'direc', 'division']

Finished loading DataFrames into SQLite3!
